# RAG with Gemini API
Complete RAG pipeline with Google Gemini for answer generation

In [ ]:
!pip install google-generativeai python-dotenv --quiet

In [ ]:
import sys
sys.path.append('../src')

from retrieval import RetrieverSystem
from llm_generator import AnswerGenerator
from sentence_transformers import SentenceTransformer

## Step 1: Load Retriever

In [ ]:
# Load retriever
retriever = RetrieverSystem.load_index(
    '../index/faiss.index',
    '../index/embeddings.npy',
    '../index/corpus_chunks.json',
    '../index/corpus_meta.json'
)

# Load embedding model
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
print("✅ Models loaded")

## Step 2: Setup Answer Generator

In [ ]:
# Initialize answer generator (loads Gemini)
generator = AnswerGenerator()
print("✅ Gemini generator initialized")

## Step 3: Create RAG Pipeline

In [ ]:
def rag_answer(query, k=10):
    """
    Complete RAG pipeline:
    1. Retrieve relevant chunks
    2. Generate answer with Gemini
    """
    # Step 1: Retrieve
    query_emb = model.encode([query])[0]
    contexts = retriever.search(query_emb, k=k)
    
    # Step 2: Generate answer
    result = generator.generate_answer(query, contexts)
    
    return result

## Step 4: Test RAG System

In [ ]:
# Test queries
test_queries = [
    "كيف أحصل على رخصة ليموزين؟",
    "ما هي خطوات تسجيل المقررات في جامعة قطر؟",
    "كيف أطلب استشارة طبية؟",
]

for query in test_queries:
    print(f"\n{'='*80}")
    print(f"السؤال: {query}")
    print('='*80)
    
    result = rag_answer(query)
    
    print(f"\nالإجابة:\n{result['answer']}")
    
    print(f"\nالمصادر:")
    for i, src in enumerate(result['sources'], 1):
        print(f"  {i}. {src['file']} ({src['category']}) - Score: {src['score']:.3f}")

## Step 5: Interactive Chat

In [ ]:
# Interactive mode
print("مساعد الخدمات الحكومية - اكتب 'خروج' للإنهاء\n")

while True:
    query = input("\nسؤالك: ")
    
    if query.strip() in ['خروج', 'exit', 'quit']:
        print("شكراً لاستخدامك المساعد!")
        break
    
    if not query.strip():
        continue
    
    result = rag_answer(query)
    print(f"\n{result['answer']}")

## ✅ Complete RAG System with Gemini

Features:
- Semantic search with embeddings
- FAISS for fast retrieval
- Gemini for natural language generation
- Arabic language support